Experiment: Concatenates together last 4 attention distribution
------
concatenates together the last four attention distributions; using only one attention head.

In [1]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

In [4]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)
#attention_dataloader = dataset.return_dataloader()
#attention_experiment = prepare_model.initialize_attention_experiment(params.intermediary_task)

06/05/2019 09:37:32 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
500it [00:00, 8284.19it/s]


In [6]:
len(dataset)

373

In [5]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label'])

In [8]:
path_label_data = params.intermediary_task['task_specific_params']['target_data']
sentence_toks = prepare_model.get_sample_toks(path_label_data)

500it [00:00, 91442.92it/s]


In [15]:
def add_word():
    """
    """
    for entry in dataset:
        idx = int(entry["index"])
        print(sentence_toks[idx])
        break

['the', 'tr', '##op', '##he', 'eric', 'bo', '##mp', '##ard', 'is', 'an', 'annual', 'elite', 'figure', 'skating', 'competition', 'which', 'takes', 'place', 'in', 'paris', ',', 'france', '.']


In [16]:
sentence_toks[1]

['the',
 'tr',
 '##op',
 '##he',
 'eric',
 'bo',
 '##mp',
 '##ard',
 'is',
 'an',
 'annual',
 'elite',
 'figure',
 'skating',
 'competition',
 'which',
 'takes',
 'place',
 'in',
 'paris',
 ',',
 'france',
 '.']

In [7]:
dataset[0]["pred_ids"] 

{'pre_ids': tensor([ 1996, 19817,  7361,  5369,  4388,  8945,  8737,  4232,  2003,  2019,
          3296,  7069,  3275, 10080,  2971,  2029,  3138,  2173,  1999,  3000,
          1010,  2605,  1012,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'masks': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8),
 'pre_lens': tensor(23),
 'post_in_ids': tensor([

In [3]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [5]:
# Extracting indices of bias labels 
intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = [label.flatten().tolist().index(1) for label in intermediary_labels]
# Extracting attention scores of biased token
biased_tok_dist = return_idx_attention_dist(attention_scores, bias_indices)
# windowing indices
windowed_data, removed_indices = window_attention_dist(biased_tok_dist, bias_indices, window_size=params.final_task['window_size'])

In [ ]:
concat_dist = concat_attention_dist(windowed_data)
print(concat_dist)

In [10]:
dataset.remove_indices(removed_indices)
dataset.add_data(torch.stack(concat_dist), "attention_dist")
dataset.shuffle_data()

Run the cell below to reset params 
-----

In [11]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [12]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 44, 'hidden_dim': 10, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [16]:
statistics

{'auc': [(0.6752843679348468, 0.8509239577269152), 0.7677060136452627],
 'accuracy': [(0.6395348837209303, 0.8023255813953488), 0.7262790697674417]}

New run 

In [35]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [36]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 11, 'hidden_dim': 3, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [22]:
statistics

{'auc': [(0.49685111400227683, 0.8255225075881711), 0.7107909485455589],
 'accuracy': [(0.5869186046511627, 0.7502906976744186), 0.6730232558139536]}

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [23]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [24]:
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [25]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [26]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

0.5602870774269103


In [27]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [28]:
print(max_auc)

0.6637993874070985


In [31]:
print(min_auc)

0.6379312454511352
